## 1. 构建 ResMicroDB corpus

In [1]:
from MiCoGPT.utils.construct import construct

input_path = "../data/try2_withCC/abundance_all_90338.csv"
output_path = "../data/try2_withCC/ResMicroDB_90338_new.pkl"
meta_path = "../data/try2_withCC/Phyloseq_ResMicroDb_metadata_merge_v2_251129.tsv"

corpus = construct(
    input_path=str(input_path),
    output_path=str(output_path),
    meta_path=str(meta_path),
    key="genus"
)


max_len = 512
tokenizer vocab size = 1121


/Users/kiancai/STA24/CWD/STAi/MiCoGPT/MiCoGPT/utils/construct.py:23: DtypeWarning: Columns (11,20) have mixed types. Specify dtype option on import or set low_memory=False.
  meta_df = pd.read_csv(meta_path, sep="\t", index_col="Run")


metadata shape = (90338, 42)
0 samples are dropped for all zeroes
Your data will be normalized with the phylogeny mean and std.


100%|██████████| 90338/90338 [00:16<00:00, 5591.01it/s]


Total 90338 samples.
            Max length is 701.
            Average length is 57.525825234120745.
            Min length is 4.
corpus length: 90338
